## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from cunfig import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_database/WeatherPy_database.csv")
city_data_df.tail()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
661,661,Norman Wells,65.2820,-126.8329,-52.60,65,40,0.96,CA,mist
662,662,Olot,42.1810,2.4901,57.00,54,54,1.99,ES,broken clouds
663,663,Constitucion,-35.3333,-72.4167,72.16,56,71,5.82,CL,broken clouds
664,664,Mhlume,-26.0333,31.8500,77.00,88,91,1.14,SZ,overcast clouds
665,665,Awbari,26.5921,12.7805,62.98,28,69,8.86,LY,broken clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Minimum temperature you would like? "))
max_temp = float(input("Maximum temperature you would like? "))

Minimum temperature you would like? 75
Maximum temperature you would like? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preffered_citites_df = city_data_df.loc[(city_data_df["Max Temp"]<= max_temp)&
                                       (city_data_df["Max Temp"]>=min_temp)]
preffered_citites_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
3,3,Rikitea,-23.1203,-134.9692,78.21,69,100,10.87,PF,NaN
5,5,Hithadhoo,-0.6000,73.0833,82.60,75,72,10.71,MV,NaN
21,21,Cayenne,4.9333,-52.3333,84.20,65,90,17.27,GF,NaN
26,26,Sabang,5.8933,95.3214,81.43,75,1,19.17,ID,NaN
29,29,Vaini,-21.2000,-175.2000,77.00,100,75,2.30,TO,NaN


In [9]:
# 4a. Determine if there are any empty rows.
preffered_citites_df.count()

Unnamed: 0             180
City                   180
Lat                    180
Lng                    180
Max Temp               180
Humidity               180
Cloudiness             180
Wind Speed             180
Country                180
Current Description    120
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
Preffered_citites_df = preffered_citites_df.dropna()
Preffered_citites_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
225,225,Rikitea,-23.1203,-134.9692,78.21,69,100,10.87,PF,overcast clouds
227,227,Hithadhoo,-0.6000,73.0833,82.60,75,72,10.71,MV,broken clouds
243,243,Cayenne,4.9333,-52.3333,84.20,65,90,17.27,GF,overcast clouds
248,248,Sabang,5.8933,95.3214,81.43,75,1,19.17,ID,clear sky
251,251,Vaini,-21.2000,-175.2000,77.00,100,75,2.30,TO,light rain


In [12]:
Preffered_citites_df = Preffered_citites_df.drop("Unnamed: 0",axis = 1)
Preffered_citites_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
225,Rikitea,-23.1203,-134.9692,78.21,69,100,10.87,PF,overcast clouds
227,Hithadhoo,-0.6000,73.0833,82.60,75,72,10.71,MV,broken clouds
243,Cayenne,4.9333,-52.3333,84.20,65,90,17.27,GF,overcast clouds
248,Sabang,5.8933,95.3214,81.43,75,1,19.17,ID,clear sky
251,Vaini,-21.2000,-175.2000,77.00,100,75,2.30,TO,light rain


In [49]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = Preffered_citites_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
225,Rikitea,PF,78.21,overcast clouds,-23.1203,-134.9692,
227,Hithadhoo,MV,82.60,broken clouds,-0.6000,73.0833,
243,Cayenne,GF,84.20,overcast clouds,4.9333,-52.3333,
248,Sabang,ID,81.43,clear sky,5.8933,95.3214,
251,Vaini,TO,77.00,light rain,-21.2000,-175.2000,
252,Butaritari,KI,80.26,light rain,3.0707,172.7902,
255,Ancud,CL,80.01,clear sky,-41.8697,-73.8203,
256,Puerto Ayora,EC,84.20,clear sky,-0.7393,-90.3518,
258,Caravelas,BR,86.20,clear sky,-17.7125,-39.2481,
260,Tiarei,PF,75.20,few clouds,-17.5333,-149.3333,


In [50]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [53]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
#6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [62]:
# 7. Drop the rows where there is no Hotel Name.
index_name = hotel_df[hotel_df["Hotel Name"]==""].index
clean_hotel_df = hotel_df.drop(index_name,inplace = True)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
629,Tautira,PF,75.20,few clouds,-17.7333,-149.1500,Pension Oaoa
632,Saint-Philippe,RE,82.40,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
633,Arlit,NE,78.42,scattered clouds,18.7369,7.3853,Case de Passage De la Prefecture
652,Isangel,VU,77.94,broken clouds,-19.5500,169.2667,Volcano Sea View Tree House
655,Viedma,AR,90.00,clear sky,-40.8135,-62.9967,Hotel Cristal


In [63]:
hotel_df.tail()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
629,Tautira,PF,75.20,few clouds,-17.7333,-149.1500,Pension Oaoa
632,Saint-Philippe,RE,82.40,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
633,Arlit,NE,78.42,scattered clouds,18.7369,7.3853,Case de Passage De la Prefecture
652,Isangel,VU,77.94,broken clouds,-19.5500,169.2667,Volcano Sea View Tree House
655,Viedma,AR,90.00,clear sky,-40.8135,-62.9967,Hotel Cristal


In [67]:
# 8a. Create the output File (CSV)
clean_hotel_df = hotel_df
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv("output_data_file.csv", index_label="City_ID")

In [68]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [70]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))